In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import tqdm
from PIL import Image

from splat.custom_pytorch_implementation.create_image_custom import (
    create_image_full_custom_multiple_gaussians_with_splat_gaussians,
)
from splat.gaussians import Gaussians

from splat.custom_pytorch_implementation.create_image_auto import (
    Camera,
    Gaussian,
    create_image_covariance_test_auto,
    create_image_full_auto,
    create_image_full_auto_multiple_gaussians,
    create_image_full_auto_multiple_gaussians_with_splat_gaussians,
)
from splat.render_engine.gaussianScene2 import GaussianScene2


def return_gaussians() -> Gaussians:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    point_3d1 = torch.tensor([[0.101, 0.1001, -4]], dtype=torch.float32, device=device).requires_grad_(
        True
    )
    r1 = torch.tensor([[0.5, 0.02, 0.03, 0.001]], dtype=torch.float32, device=device).requires_grad_(
        True
    )
    s1 = torch.tensor([[0.1, 0.15, 0.2]], dtype=torch.float32, device=device).requires_grad_(True)
    color1 = torch.tensor([0.4, 0.6, 0.8], dtype=torch.float32, device=device).requires_grad_(True)
    opacity1 = torch.tensor([0.5], dtype=torch.float32, device=device).requires_grad_(True)

    point_3d2 = torch.tensor(
        [[0.0999, 0.0999, -4.1]], dtype=torch.float32, device=device
    ).requires_grad_(True)
    r2 = torch.tensor([[0.2, 0.04, 0.03, 0.001]], dtype=torch.float32, device=device).requires_grad_(
        True
    )
    s2 = torch.tensor([[0.2, 0.16, 0.1]], dtype=torch.float32, device=device).requires_grad_(True)
    color2 = torch.tensor([0.1, 0.15, 0.2], dtype=torch.float32, device=device).requires_grad_(True)
    opacity2 = torch.tensor([0.9], dtype=torch.float32, device=device).requires_grad_(True)

    return Gaussians(
        points=torch.stack([point_3d1, point_3d2]).squeeze(1),
        colors=torch.stack([color1, color2]).squeeze(1),
        scales=torch.stack([s1, s2]).squeeze(1),
        quaternions=torch.stack([r1, r2]).squeeze(1),
        opacity=torch.stack([opacity1, opacity2]).squeeze(1),
        requires_grad=True
    )


def return_camera() -> Camera:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    focal_x = torch.tensor([100.0], device=device)
    focal_y = torch.tensor([100.0], device=device)
    width = 32
    height = 32
    camera_rotation = torch.tensor([1, 0, 0, 0], device=device).unsqueeze(0)
    camera_translation = torch.tensor([[-0.1, -0.1, 0.0]], device=device)

    return Camera(
        focal_x=focal_x,
        focal_y=focal_y,
        c_x=0.0,
        c_y=0.0,
        width=width,
        height=height,
        camera_rotation=camera_rotation,
        camera_translation=camera_translation,
        device=device
    )


def return_gt_image() -> torch.Tensor:
    gt_image = Image.open("gt.png")
    gt_image = np.array(gt_image) / 255.0
    return torch.tensor(gt_image)


def preprocess_for_gaussian_scene(camera: Camera, gaussian_scene: GaussianScene2) -> Gaussians:
    extrinsic_matrix = camera.extrinsic_matrix
    intrinsic_matrix = camera.intrinsic_matrix
    focal_x = camera.focal_x
    focal_y = camera.focal_y
    width = camera.width
    height = camera.height
    tile_size = 16
    preprocessed_gaussians = gaussian_scene.preprocess(
        extrinsic_matrix=extrinsic_matrix,
        intrinsic_matrix=intrinsic_matrix,
        focal_x=focal_x,
        focal_y=focal_y,
        width=width,
        height=height,
        tile_size=tile_size,
    )
    return preprocessed_gaussians


/home/da2986/intro_to_gaussian_splatting/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
camera = return_camera()
gaussians = return_gaussians()
# gt_image = return_gt_image()

output_auto1 = create_image_full_auto_multiple_gaussians_with_splat_gaussians(
    camera, gaussians, camera.height, camera.width
)
# output_custom = create_image_full_custom_multiple_gaussians_with_splat_gaussians(
#     camera, gaussians, camera.height, camera.width
# )
# print(torch.allclose(output_auto1, output_custom))

gaussian_scene = GaussianScene2(gaussians=gaussians)
preprocessed_gaussians = preprocess_for_gaussian_scene(camera, gaussian_scene)
output_scene = gaussian_scene.render_cuda(
    preprocessed_gaussians=preprocessed_gaussians,
    height=camera.height,
    width=camera.width,
    tile_size=16,
)
print(torch.allclose(output_auto1, output_scene))

cov_3d tensor([[[ 0.0104, -0.0003,  0.0035],
         [-0.0003,  0.0226, -0.0014],
         [ 0.0035, -0.0014,  0.0395]]], device='cuda:0',
       grad_fn=<ViewBackward0>)
cov_3d tensor([[[ 0.0375,  0.0026, -0.0078],
         [ 0.0026,  0.0235,  0.0049],
         [-0.0078,  0.0049,  0.0146]]], device='cuda:0',
       grad_fn=<ViewBackward0>)


/home/da2986/intro_to_gaussian_splatting/splat/custom_pytorch_implementation/auto_functions.py:109: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return final_color, torch.tensor(current_T)


covariance3d tensor([[[ 0.0104, -0.0003,  0.0035],
         [-0.0003,  0.0226, -0.0014],
         [ 0.0035, -0.0014,  0.0395]],

        [[ 0.0375,  0.0026, -0.0078],
         [ 0.0026,  0.0235,  0.0049],
         [-0.0078,  0.0049,  0.0146]]], device='cuda:0',
       grad_fn=<UnsafeViewBackward0>)
means tensor([[27.6093, 16.7109, -3.0000],
        [14.3186, 14.3186, -3.1000]], device='cuda:0', grad_fn=<CopySlices>)
color tensor([[0.4000, 0.6000, 0.8000],
        [0.1000, 0.1500, 0.2000]], device='cuda:0', grad_fn=<IndexBackward0>)
opacity tensor([0.5000, 0.9000], device='cuda:0', grad_fn=<IndexBackward0>)
inverted_covariance_2d tensor([[[ 0.1467,  0.0017],
         [ 0.0017,  0.0693]],

        [[ 0.0445, -0.0048],
         [-0.0048,  0.0706]]], device='cuda:0', grad_fn=<CopySlices>)
False
